In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import json
#import utm as utm

#*********basic restaurant data
restaurant = pd.read_csv("restaurants.csv")

# restaurant.dropna(
#     axis=0,
#     how='any',
#     thresh=None,
#     subset=None,
#     inplace=True
# )
restaurant.dropna()

color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(restaurant, 
                        lat="LATITUDE", 
                        lon="LONGITUDE", 
                        hover_name="NAME", 
                        hover_data=["PHYSICALADDRESS", "PHYSICALCITY"],
                        # color="Listed",
                        # color_continuous_scale=color_scale,
                        zoom=9, 
                        height=300)


# fig = go.Figure(go.Scattermapbox())

#lat_lon = utm.to_latlon(508009.1730000004, 5446008.925249999, 10, 'U')
#print(lat_lon)

# fig = px.scatter_mapbox(restaurant, lat="Lat", lon="Long", hover_name="Address", hover_data=["Address", "Listed"],
#                         color_discrete_sequence=["fuchsia"], zoom=3, height=300)


fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [4]:
import plotly.express as px
import pandas as pd

# Create two Pandas DataFrames containing the location data for each set of locations
df1 = pd.DataFrame({'lat': [40.7128, 51.5074, 35.6895],
                    'lon': [-74.0060, -0.1278, 139.6917],
                    'group': ['group1', 'group1', 'group1']})

df2 = pd.DataFrame({'lat': [52.5200, 37.7749, 48.8566],
                    'lon': [13.4050, -122.4194, 2.3522],
                    'group': ['group2', 'group2', 'group2']})

# Concatenate the two DataFrames into a single DataFrame
df = pd.concat([df1, df2])

# Create the scatter plot using Plotly Express
fig = px.scatter_mapbox(df, lat='lat', lon='lon', color='group', zoom=2)

# Show the plot
fig.show()


In [ ]:
import json
from types import SimpleNamespace
#*********basic transit data
#transit_json = open('dataset/transit.json')

with open('dataset/transit.json') as your_file:
    your_dict = json.load(your_file)
    your_file.seek(0)
    your_root = json.load(your_file, object_hook= lambda x: 
                                        SimpleNamespace(**x))
type(your_root[2])
# transit = json.load(transit_json)
# for i in transit[2]:
#     print(i)

In [1]:
from scripts.data_processing import csv_to_restaurant_data, generate_clean_restaurant

transit_stop_array_raw = []

csv_to_restaurant_data(transit_stop_array_raw)

print(len(transit_stop_array_raw))

generate_clean_restaurant(transit_stop_array_raw)

for i in range(5):
    transitStopEle = transit_stop_array_raw[i]
    print(f"Stop {i+1}:")
    print(f"tracking_number: {transitStopEle.tracking_number}")
    print(f"name: {transitStopEle.name}")
    print(f"x: {transitStopEle.utm_x}")
    print(f"y: {transitStopEle.utm_y}")
    print(f"address: {transitStopEle.address}")
    print(f"lon: {transitStopEle.lon}")
    print(f"lat: {transitStopEle.lat}")
    print("----------------")



1856
Stop 1:
tracking_number: A013601    
name: Brownsville Bar Marine Pub
x: 507888.9494818421
y: 5450090.612264993
address: 11940 Old Yale Rd
lon: -122.89170074
lat: 49.20355988
----------------
Stop 2:
tracking_number: HCAR-C93N7W
name: Sirka Gourmet Take Out
x: 508004.2750171515
y: 5442323.219720248
address: 7182 120th St
lon: -122.89027207
lat: 49.13368808
----------------
Stop 3:
tracking_number: HCAR-BZ7TNG
name: Westin Pizza
x: 508015.4270343689
y: 5442319.247173271
address: 7182 120th St
lon: -122.89011927
lat: 49.1336522
----------------
Stop 4:
tracking_number: A019941    
name: De Dutch Pannekoek House #25
x: 508023.27059128904
y: 5442556.686866164
address: 7310 120 St
lon: -122.89000702
lat: 49.13578796
----------------
Stop 5:
tracking_number: SODN-65HUKF
name: Panago #10
x: 508015.18404021097
y: 5445061.253707203
address: 8556 120 St
lon: -122.89006805
lat: 49.15831757
----------------


In [1]:
from scripts.data_processing import json_to_stop_data, generate_clean_transit_stop

transit_stop_array_raw = []

json_to_stop_data(transit_stop_array_raw)

print(len(transit_stop_array_raw))

generate_clean_transit_stop(transit_stop_array_raw)

for i in range(5):
    transitStopEle = transit_stop_array_raw[i]
    print(f"Stop {i+1}:")
    print(f"id: {transitStopEle.id}")
    print(f"x: {transitStopEle.utm_x}")
    print(f"y: {transitStopEle.utm_y}")
    print(f"city: {transitStopEle.city}")
    print(f"lon: {transitStopEle.lon}")
    print(f"lat: {transitStopEle.lat}")
    print("----------------")



3012
Stop 1:
id: 59741
x: 502265.75100000016
y: 5450336.422
city: BURNABY
lon: -122.96889442511298
lat: 49.20581759492203
----------------
Stop 2:
id: 55179
x: 502646.13989999983
y: 5437429.9046
city: DELTA
lon: -122.96375700032506
lat: 49.089716999975494
----------------
Stop 3:
id: 54877
x: 502761.76400000043
y: 5437393.789999999
city: DELTA
lon: -122.96217359567265
lat: 49.08939162394247
----------------
Stop 4:
id: 52367
x: 502562.2479999997
y: 5447647.1559999995
city: RICHMOND
lon: -122.96484108746395
lat: 49.181625641889404
----------------
Stop 5:
id: 52771
x: 503023.61400000006
y: 5451020.790999999
city: BURNABY
lon: -122.95848488033786
lat: 49.21197041998329
----------------


In [2]:
from scripts.data_processing import map_system_converter, GeoSystem

x, y = map_system_converter(GeoSystem.UTM_SURREY, GeoSystem.WGS84, 502265.75100000016,5450336.422)
print(x, y)


49.20581759492203 -122.96889442511298


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

import pandas as pd
import numpy as np

df1 = gpd.GeoDataFrame(
    {'geometry': [Point(xy) for xy in zip(lon_array, lat_array)]},
    crs="EPSG:4326")
df2 = gpd.GeoDataFrame(
    {'geometry': [Point(xy) for xy in zip(lon_array, lat_array)]},
    crs="EPSG:4326")

df1['x'] = df1.geometry.x
df1['y'] = df1.geometry.y
df2['x'] = df2.geometry.x
df2['y'] = df2.geometry.y

# Merge the two GeoDataFrames on the x and y columns
merged = pd.merge(df1[['x', 'y']], df2[['x', 'y']], on=['x', 'y'])

# Calculate the correlation between the two sets of coordinates
correlation = np.corrcoef(merged.x, merged.y)[0, 1]
print(f"The correlation between the two GeoDataFrames is: {correlation}")
#
# df1.to_file("../data/processed/data.shp", driver="ESRI Shapefile")

# w = ps.weights.Queen.from_dataframe(df1)
# cij = ps_explore.spatial_covariance(df1['geometry'], df2['geometry'], w, transform='r')

# ps_explore.plot_cis(cij)



In [2]:
import pandas as pd
import plotly.express as px

df_restaurant = pd.read_csv("../data/processed/restaurant.csv")
df_transit = pd.read_csv("../data/processed/transit_stops.csv")

df_restaurant_plot = df_restaurant[["NAME", "CITY", "LONGITUDE", "LATITUDE"]].copy()
df_transit_plot = df_transit[["ID", "CITY", "LONGITUDE", "LATITUDE"]].copy()

df_restaurant_plot = df_restaurant_plot.rename(columns={"NAME":"NAME/ID"})
df_transit_plot = df_transit_plot.rename(columns={"ID":"NAME/ID"})

df_restaurant_plot['COLOR'] = 'red'
df_transit_plot['COLOR'] = 'green'

df_plot = pd.concat([df_restaurant_plot, df_transit_plot])

fig = px.scatter_mapbox(df_plot, lat="LATITUDE", lon="LONGITUDE", hover_name="NAME/ID", hover_data=["NAME/ID", "CITY"],
                    color_discrete_sequence=[df_plot.Color], zoom=9, height=300)


NameError: name 'px' is not defined